In [1]:
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = 'data/train'
validation_dir = 'data/valid'
test_dir = 'data/test'

In [3]:
# Define data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Load and preprocess the data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3615 images belonging to 30 classes.
Found 150 images belonging to 30 classes.
Found 150 images belonging to 30 classes.


In [5]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Train the model with data augmentation
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size
)

Epoch 1/50
112/112 [==============================] - 44s 383ms/step - loss: 3.0592 - accuracy: 0.1549 - val_loss: 2.3542 - val_accuracy: 0.3200
Epoch 2/50
112/112 [==============================] - 44s 384ms/step - loss: 2.1748 - accuracy: 0.3820 - val_loss: 2.1425 - val_accuracy: 0.3667
Epoch 3/50
112/112 [==============================] - 44s 389ms/step - loss: 1.8010 - accuracy: 0.4852 - val_loss: 1.4785 - val_accuracy: 0.5533
Epoch 4/50
112/112 [==============================] - 44s 386ms/step - loss: 1.5270 - accuracy: 0.5571 - val_loss: 1.4725 - val_accuracy: 0.5133
Epoch 5/50
112/112 [==============================] - 44s 387ms/step - loss: 1.3681 - accuracy: 0.5986 - val_loss: 1.2470 - val_accuracy: 0.6067
Epoch 6/50
112/112 [==============================] - 44s 389ms/step - loss: 1.2215 - accuracy: 0.6454 - val_loss: 1.1031 - val_accuracy: 0.6600
Epoch 7/50
112/112 [==============================] - 44s 385ms/step - loss: 1.0812 - accuracy: 0.6716 - val_loss: 1.2508 - val_ac

In [7]:
# Evaluate the model on the test set
scores = model.evaluate(test_generator, verbose=0)
print("Test accuracy: %.2f%%" % (scores[1]*100))

Test accuracy: 72.67%


In [8]:
# Save the model to a file
model.save('ball_classifier.h5')